# Big Data HS 2023

## JSONiq tutorial - weeks 4 and 5

This is the JSONiq tutorial for weeks 4 and 5.

Do not forget to use localhost:8888 as the URL to make sure the notebook is accessed via docker! And if it does not work, you should delete all containers, images, and volumes, then try again with



````
docker-compose up
````

Like last week, junst run the cell below to connect the Jupyter notebook with RumbleDB.

In [1]:
%load_ext rumbledb
%env RUMBLEDB_SERVER=http://localhost:9090/jsoniq

env: RUMBLEDB_SERVER=http://localhost:9090/jsoniq


## Arrays and Objects
Any well-formed JSON document is a JSONiq expression as well. This means that you can copy-and-paste any JSON document into a query, including arrays and objects. 

In [2]:
%%jsoniq
[ 2, 3, 5, 7, 11, 13 ]

Took: 0.03573894500732422 ms
[2, 3, 5, 7, 11, 13]


In [3]:
%%jsoniq
{ "pi" : 3.14, "sq2" : 1.4 }

Took: 0.022755146026611328 ms
{"pi": 3.14, "sq2": 1.4}


Just like in a JSON document, you can arbitrarily nest arrays and objects.

In [4]:
%%jsoniq
{
  "operations" : [
    { "binary" : [ "and", "or"] },
    { "unary" : ["not"] }
  ],
  "bits" : [
    0, 1
  ]
}

Took: 0.010058164596557617 ms
{"operations": [{"binary": ["and", "or"]}, {"unary": ["not"]}], "bits": [0, 1]}


In [5]:
%%jsoniq
[ { "Question" : "Ultimate" }, ["Life", "the universe", "and everything"] ]

Took: 0.008957147598266602 ms
[{"Question": "Ultimate"}, ["Life", "the universe", "and everything"]]


The above queries are "idempotent" (they just output themselves). This works with objects, arrays (even nested), strings, numbers, booleans, null.

It also works the other way round: if your query outputs an object, you can use it as a JSON document.
JSONiq is a declarative language. This means that you only need to say what you want - the compiler will take care of the how. 

In the above queries, you are basically saying: I want to output this JSON content, and here it is.

In fact JSONiq makes JSON "dynamic": try to replace numbers with arithmetic formulas, keys with concatenations of strings, etc and see how the resulting JSON object is automatically created.

In [6]:
%%jsoniq
[{
    "foo" : 2 + 2,
    "bar" : if(2 gt 1) then true else false
}, 5 * 7]

Took: 0.01450967788696289 ms
[{"foo": 4, "bar": true}, 35]


You can also use the `to` operator to generate an array.

In [7]:
%%jsoniq
[1 to 5]

Took: 0.009694814682006836 ms
[1, 2, 3, 4, 5]



## Sequences

Until now, we have only been working with single values (an object, an array, a number, a string, a boolean). JSONiq supports sequences of values. You can build a sequence using commas:


In [8]:
%%jsoniq
 (1, 2, 3, 4, 5, 6, 7, 8, 9, 10)

Took: 0.01904892921447754 ms
1
2
3
4
5
6
7
8
9
10


Sequences can be heterogeneous.

In [9]:
%%jsoniq
1, true, 4.2e1, "Life", [1,2], {"foo": "bar"}

Took: 0.009652853012084961 ms
1
true
42
"Life"
[1, 2]
{"foo": "bar"}


You can also use the `to` operator to generate a sequence.

In [10]:
 %%jsoniq
1 to 5

Took: 0.009476184844970703 ms
1
2
3
4
5


Unlike arrays and objects, sequences are flat, which means sequences cannot be nested. The following cells are the same.

In [11]:
%%jsoniq
 (1, 2), ((3)), ((4, (5)), 6, 7, 8, 9), 10

Took: 0.009335994720458984 ms
1
2
3
4
5
6
7
8
9
10


In [12]:
%%jsoniq
 (1, 2, 3, 4, 5, 6, 7, 8, 9, 10)

Took: 0.00943899154663086 ms
1
2
3
4
5
6
7
8
9
10


In [13]:
 %%jsoniq
1, 2, 3, 4, 5, 6, 7, 8, 9, 10

Took: 0.009459972381591797 ms
1
2
3
4
5
6
7
8
9
10


In [14]:
 %%jsoniq
1 to 10

Took: 0.010783910751342773 ms
1
2
3
4
5
6
7
8
9
10



## Lookup and Unboxing

You can lookup a specific position of an array with the `[[]]` array lookup syntax. Note that JSONiq uses the 1-based indexing.

In [15]:
 %%jsoniq
[1, 5, 9][[2]]

Took: 0.010334014892578125 ms
5


Similarly, you can lookup a specific position of a sequence with the `[]` sequence lookup syntax.

In [16]:
 %%jsoniq
(1, 5, 9)[2]

Took: 0.009680032730102539 ms
5


Moreover, you can filter values in a sequence satisfying certain conditions. The result of the predicate is converted to a boolean. $$ is used to denote the current item being tested by the predicate.

In [17]:
 %%jsoniq
(1 to 10)[$$ mod 2 = 0]

Took: 0.009526968002319336 ms
2
4
6
8
10


In [18]:
 %%jsoniq
(1 to 10)[$$ gt 3]

Took: 0.010287284851074219 ms
4
5
6
7
8
9
10


You can expand an array to a sequence using the `[]` array unboxing syntax.

In [19]:
%%jsoniq
[1, 2, 3][]

Took: 0.010519027709960938 ms
1
2
3


You can also lookup a key-value pair in an object with the `.` object lookup syntax.

In [20]:
%%jsoniq
let $x := { "pi" : 3.14, "sq2" : 1.4 }
return $x.sq2

Took: 0.009566783905029297 ms
1.4


If you apply the dot object-lookup syntax to a sequence of objects, you get a sequence of corresponding values. In fact, the same apply to array lookup and array unboxing, which work on sequences of arrays as well.

In [21]:
%%jsoniq
let $x := (
     { "foo" : 1 },
     { "foo" : 2 },
     { "foo" : 1 },
     { "bar" : 1 }
    )
return $x.foo

Took: 0.01721334457397461 ms
1
2
1


You can chain lookups, unboxing, and boxing arbitrarily.

In [22]:
%%jsoniq
let $x := 
{
  "operations" : [
    { "binary" : [ "and", "or"] },
    { "unary" : ["not"] }
  ],
  "bits" : [
    0, 1
  ]
}
return ([$x.operations[].binary[[2]]], $x.bits[[2]], $x[[]].operations)

Took: 0.010958194732666016 ms
["or"]
1
[{"binary": ["and", "or"]}, {"unary": ["not"]}]


# Try your own queries!

This notebook is interactive. You can edit all queries above and also execute your own! We will show you more features every week.

In [23]:
%%jsoniq
[ 1 + 1 ]

Took: 0.008665084838867188 ms
[2]


In [24]:
%%jsoniq
{ "foo": 1 }

Took: 0.011269807815551758 ms
{"foo": 1}


In [25]:
%%jsoniq
1, 2, 3

Took: 0.011321067810058594 ms
1
2
3


In [26]:
%%jsoniq
1+1

Took: 0.010106086730957031 ms
2


In [27]:
%%jsoniq
1+1

Took: 0.009326934814453125 ms
2


In [28]:
%%jsoniq
1+1

Took: 0.009218931198120117 ms
2


In [ ]:
%%jsoniq


In [ ]:
%%jsoniq
